In [ ]:
import os
import re
import string
import random
from glob import glob
from argparse import Namespace

import tensorflow as tf
print(tf.__version__)
from tensorflow.keras import layers
from tensorflow.keras import models
from keras.optimizers import *
from keras.models import Model
from keras.layers import *
from keras.layers.core import *
from keras.layers.convolutional import *
from keras import backend as K
import keras
import tensorflow as tf

In [ ]:
import wandb
from wandb.keras import WandbMetricsLogger
from wandb.keras import WandbModelCheckpoint

In [ ]:
def id_generator(size=6, chars=string.ascii_uppercase + string.digits):
    return ''.join(random.choice(chars) for _ in range(size))

random_id = id_generator(size=8)
print('Experiment Id: ', random_id)


configs = Namespace(
    num_frames = 32,
    batch_size = 128,
    epochs = 60,
    resizing_interpolation = "nearest",
    learning_rate = 1e-3,
    num_steps = 1.0,
    experiment_id = random_id,
    num_jcd_features = 3321, # ((82*82)-82)/2
    num_classes = 250,
    filters = 256,
)


LIP = [
    61, 185, 40, 39, 37, 0, 267, 269, 270, 409,
    291, 146, 91, 181, 84, 17, 314, 405, 321, 375,
    78, 191, 80, 81, 82, 13, 312, 311, 310, 415,
    95, 88, 178, 87, 14, 317, 402, 318, 324, 308,
]

RIGHT_EYE = [
    246, 161, 160, 159, 158, 157, 173,
    33, 7, 163, 144, 145, 153, 154, 155, 133,
    247, 30, 29, 27, 28, 56, 190,
    130, 25, 110, 24, 23, 22, 26, 112, 243,
    113, 225, 224, 223, 222, 221, 189,
    226, 31, 228, 229, 230, 231, 232, 233, 244,
    143, 111, 117, 118, 119, 120, 121, 128, 245,
]

LEFT_EYE = [
    466, 387, 386, 385, 384, 398,
    263, 249, 390, 373, 374, 380, 381, 382, 362,
    467, 260, 259, 257, 258, 286, 414,
    359, 255, 339, 254, 253, 252, 256, 341, 463,
    342, 445, 444, 443, 442, 441, 413,
    446, 261, 448, 449, 450, 451, 452, 453, 464,
    372, 340, 346, 347, 348, 349, 350, 357, 465,
]

In [ ]:
data_path = "../data/tfrecords"


def natural_keys(text):
    ""
    def atoi(text):
        return int(text) if text.isdigit() else text
    
    return [atoi(c) for c in re.split(r'(\d+)', text)]

tfrecords = sorted(glob(f"{data_path}/*.tfrec"), key=natural_keys)
len(tfrecords)

In [ ]:
train_tfrecords, valid_tfrecords = tfrecords[:19], tfrecords[19:]

def parse_sequence(serialized_sequence):
    return tf.io.parse_tensor(
        serialized_sequence,
        out_type=tf.float32,
    )


def parse_tfrecord_fn(example):
    feature_description = {
        "n_frames": tf.io.FixedLenFeature([], tf.float32),
        "frames": tf.io.FixedLenFeature([], tf.string),
        "label": tf.io.FixedLenFeature([], tf.int64),
    }
    
    return tf.io.parse_single_example(example, feature_description)


@tf.function
def preprocess_frames(frames):
    """
    In this preprocessing function:
    - Fill NaN values to 0.
    - Use `tf.image.resize` to interpolate.
    """
    frames = tf.where(tf.math.is_nan(frames), 0.0, frames)

    frames = tf.image.resize(
        frames, (configs.num_frames, 543), method=configs.resizing_interpolation
    )

    return frames


def parse_data(example):
    # Parse Frames
    n_frames = example["n_frames"]
    frames = tf.reshape(parse_sequence(example["frames"]), shape=(n_frames, 543, 3))
    frames = preprocess_frames(frames)
    
    # Parse Labels
    label = tf.one_hot(example["label"], depth=250)

    return frames, label


AUTOTUNE = tf.data.AUTOTUNE

train_ds = tf.data.TFRecordDataset(train_tfrecords)
valid_ds = tf.data.TFRecordDataset(valid_tfrecords)

trainloader = (
    train_ds
    .map(parse_tfrecord_fn, num_parallel_calls=AUTOTUNE)
    .shuffle(1024)
    .map(parse_data, num_parallel_calls=AUTOTUNE)
    .batch(128)
    .prefetch(AUTOTUNE)
)

validloader = (
    valid_ds
    .map(parse_tfrecord_fn, num_parallel_calls=AUTOTUNE)
    .map(parse_data, num_parallel_calls=AUTOTUNE)
    .batch(128)
    .prefetch(AUTOTUNE)
)

# JCD

In [ ]:
def compute_JCD(S_k):
    # S_k shape is (num_frames, num_joints, depth)
    # Compute pairwise euclidean distance 
    d = tf.reduce_sum((tf.expand_dims(S_k, 3)-tf.expand_dims(S_k, 2))**2,4)
    d = tf.sqrt(d)
    
    # Get lower triangle to reduce redundancy
    d = tf.linalg.LinearOperatorLowerTriangular(d).to_dense()
    
    # Extract and flatten lower triangle
    ones = tf.ones_like(d)
    mask_a = tf.linalg.band_part(ones, -1, 0) # Lower triangle mask of 0s and 1s
    mask_b = tf.linalg.band_part(ones, 0, 0)  # Diagonal mask of 0s and 1s
    mask = tf.cast(mask_a - mask_b, dtype=tf.bool) # Desired mask

    jcd = tf.boolean_mask(d, mask)
    
    return jcd

# Motion

M_k_slow = S_k_plus_1 - S_k (k = 1,2,3,...,K-1)

[(S_2-S_1), (S_3-S_2), .....]

M_k_fast = S_k_plus_2 - S_k (k = 1,3,5.....,K-2)

[(S_3-S_1), (S_5-S_3), .....]


S_k is the frame. S_k_plus_1 and S_k_plus_2 are behind/ahead S_k resp.

Part of DDNet model.

In [ ]:
def joints_diff(x):
    H, W = x.get_shape()[1], x.get_shape()[2]
    x = tf.subtract(x[:, 1:, ...], x[:, :-1, ...])
    x = tf.image.resize(x, size=[H, W], method="nearest")
    return x


def joints_motion(P, frame_l):
    P_diff_slow = Lambda(lambda x: joints_diff(x))(P)
    P_diff_slow = Reshape((frame_l, -1), name="slow_reshape")(P_diff_slow)

    P_fast = Lambda(lambda x: x[:, ::2, ...])(P)
    P_diff_fast = Lambda(lambda x: joints_diff(x))(P_fast)
    P_diff_fast = Reshape((int(frame_l / 2), -1), name="fast_reshape")(P_diff_fast)

    return P_diff_slow, P_diff_fast

# Model

In [ ]:
def get_jcd_embedding(x, filters, num_jcd_features):
    # Project down
    x = EinsumDense("abc,cd->abd", output_shape=(None, num_jcd_features//4), bias_axes="d")(x)
    x = LayerNormalization()(x)
    x = SpatialDropout1D(0.2)(x)
    x = EinsumDense("abc,cd->abd", output_shape=(None, num_jcd_features//8), bias_axes="d")(x)
    x = LayerNormalization()(x)
    x = SpatialDropout1D(0.2)(x)
    

    # Local relationship between sequences 
    x = Conv1D(filters, 3, strides=1, dilation_rate=1, padding="same")(x)
    x = LayerNormalization()(x)
    x = ReLU()(x)
    x = SpatialDropout1D(0.1)(x)
    
    x = Conv1D(filters, 3, strides=2, dilation_rate=1, padding="same")(x)
    x = LayerNormalization()(x)
    x = ReLU()(x)
    x = SpatialDropout1D(0.1)(x)
    
    x = Conv1D(filters, 3, strides=2, dilation_rate=1, padding="same")(x)
    x = LayerNormalization()(x)
    x = ReLU()(x)
    x = SpatialDropout1D(0.1)(x)

    return x


def get_slow_motion_embedding(x, filters):    
    # Local relationship between sequences 
    x = Conv1D(filters, 3, strides=1, dilation_rate=1, padding="same")(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    x = SpatialDropout1D(0.1)(x)
    
    x = Conv1D(filters, 3, strides=2, dilation_rate=1, padding="same")(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    x = SpatialDropout1D(0.1)(x)
    
    x = Conv1D(filters, 3, strides=2, dilation_rate=1, padding="same")(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    x = SpatialDropout1D(0.1)(x)
    
    return x


def get_fast_motion_embedding(x, filters):    
    # Local relationship between sequences 
    x = Conv1D(filters, 3, strides=1, dilation_rate=1, padding="same")(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    x = SpatialDropout1D(0.1)(x)
    
    x = Conv1D(filters, 3, strides=2, dilation_rate=1, padding="same")(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    x = SpatialDropout1D(0.1)(x)
    
    x = Conv1D(filters, 3, strides=1, dilation_rate=1, padding="same")(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    x = SpatialDropout1D(0.1)(x)
    
    return x

In [ ]:
def get_DDNet(num_jcd_features, filters, num_frames):
    inputs = Input(shape=(num_frames, 543, 3))
    
    lip_inputs = tf.gather(inputs, indices=LIP, axis=2)
    left_hand_inputs = inputs[:, :, 468:489, :]
    right_hand_inputs = inputs[:, :,522:,:]
    joints = tf.keras.layers.Concatenate(axis=2)(
        [lip_inputs, right_hand_inputs, left_hand_inputs]
    )
    
    
    # JCD
    jcd = compute_JCD(joints)
    jcd = tf.reshape(jcd, (-1, num_frames, num_jcd_features))
    jcd_embedding = get_jcd_embedding(jcd, 256, num_jcd_features)
    
    # Slow and Fast Motion
    slow_motion, fast_motion = joints_motion(joints, num_frames)
    slow_embedding = get_slow_motion_embedding(slow_motion, 256)
    fast_embedding = get_fast_motion_embedding(fast_motion, 256)
    
    # Merge embeddings
    x = Concatenate()([jcd_embedding, slow_embedding, fast_embedding])
    
    # Learn from the embedding
    x = Conv1D(256, 3, strides=2, dilation_rate=1, padding="same")(x)
    x = LayerNormalization()(x)
    x = ReLU()(x)
    x = SpatialDropout1D(0.1)(x)
    
    x = Conv1D(256, 3, strides=2, dilation_rate=1, padding="same")(x)
    x = LayerNormalization()(x)
    x = ReLU()(x)
    x = SpatialDropout1D(0.1)(x)
    
    x = GlobalAveragePooling1D()(x)
    
    x = Dense(250, activation="softmax")(x)

    model = Model(inputs=inputs, outputs=x)
    return model

In [ ]:
def get_JCDNet(num_jcd_features, filters, num_frames):
    inputs = Input(shape=(num_frames, 543, 3))
    
    lip_inputs = tf.gather(inputs, indices=LIP, axis=2)
    left_hand_inputs = inputs[:, :, 468:489, :]
    right_hand_inputs = inputs[:, :,522:,:]
    joints = tf.keras.layers.Concatenate(axis=2)(
        [lip_inputs, right_hand_inputs, left_hand_inputs]
    )
    
    # JCD
    jcd = compute_JCD(joints)
    jcd = tf.reshape(jcd, (-1, num_frames, num_jcd_features))
    jcd_embedding = get_jcd_embedding(jcd, 256, num_jcd_features)
    
    # Learn from the embedding
    x = Conv1D(256, 3, strides=2, dilation_rate=1, padding="same")(jcd_embedding)
    x = LayerNormalization()(x)
    x = ReLU()(x)
    x = SpatialDropout1D(0.3)(x)

    x = Conv1D(256, 3, strides=2, dilation_rate=1, padding="same")(x)
    x = LayerNormalization()(x)
    x = ReLU()(x)
    x = SpatialDropout1D(0.3)(x)
    
    x = GlobalAveragePooling1D()(x)
    
    x = Dense(250, activation="softmax")(x)

    model = Model(inputs=inputs, outputs=x)
    return model

In [ ]:
def get_MotionNet(num_jcd_features, filters, num_frames):
    inputs = Input(shape=(num_frames, 543, 3))
    
    lip_inputs = tf.gather(inputs, indices=LIP, axis=2)
    left_hand_inputs = inputs[:, :, 468:489, :]
    right_hand_inputs = inputs[:, :,522:,:]
    joints = tf.keras.layers.Concatenate(axis=2)(
        [lip_inputs, right_hand_inputs, left_hand_inputs]
    )
    
    # Slow and Fast Motion
    slow_motion, fast_motion = joints_motion(joints, num_frames)
    slow_embedding = get_slow_motion_embedding(slow_motion, 256)
    fast_embedding = get_fast_motion_embedding(fast_motion, 256)
    
    # Merge embeddings
    x = Concatenate()([slow_embedding, fast_embedding])
    
    # Learn from the embedding
    x = Conv1D(256, 3, strides=2, dilation_rate=1, padding="same")(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    x = SpatialDropout1D(0.3)(x)
    
    x = Conv1D(256, 3, strides=2, dilation_rate=1, padding="same")(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    x = SpatialDropout1D(0.3)(x)
    
    x = GlobalAveragePooling1D()(x)

    x = Dense(250, activation="softmax")(x)

    model = Model(inputs=inputs, outputs=x)
    return model

In [ ]:
tf.keras.backend.clear_session()

model = get_JCDNet(
    num_jcd_features=configs.num_jcd_features,
    filters=configs.filters,
    num_frames=configs.num_frames
)
    
# total_steps = 585*configs.epochs
# decay_steps = total_steps*configs.num_steps

# cosine_decay_scheduler = tf.keras.optimizers.schedules.CosineDecay(
#     initial_learning_rate = configs.learning_rate,
#     decay_steps = decay_steps,
#     alpha=0.1
# )

model.compile(
    tf.keras.optimizers.Adam(1e-3),
    "binary_crossentropy",
    metrics=["acc"]
)

model.summary()

In [ ]:
tf.keras.utils.plot_model(model, show_shapes=True, show_layer_names=True, expand_nested=True)

In [ ]:
run = wandb.init(
    project="kaggle-asl",
    name=f"ddnet-{configs.experiment_id}",
    config=configs,
    job_type="train",
)

earlystopper = tf.keras.callbacks.EarlyStopping(
    monitor="val_loss",
    patience=8,
    verbose=0,
    mode="auto",
    restore_best_weights=True,
)

callbacks = [
#     earlystopper,
    WandbMetricsLogger(log_freq=2),
#     WandbModelCheckpoint(
#         filepath=f"../models/ddnet-{configs.experiment_id}",
#         save_best_only=True,
#     ),
]

model.fit(
    trainloader,
    epochs=configs.epochs,
    validation_data=validloader,
    callbacks=callbacks,
)

In [ ]:
eval_loss, eval_acc = model.evaluate(validloader)
wandb.log({"eval_loss": eval_loss, "eval_acc": eval_acc})

run.finish()